# Cryptopunks: obtener datos de un cryptopunk

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

import requests
from bs4 import BeautifulSoup

import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.common.keys import Keys
from scrapy import Selector
import math

import mysql.connector

import re

## Establecer conexión con la BD

In [2]:
# Connect to the new database
mydb = mysql.connector.connect(
  host="localhost",
  user="cecilia",
  password="cecilia",
  database="cryptopunks"
)

mycursor = mydb.cursor()



## Función: Obtener la información del cryptopunk

In [3]:
def obtener_cryptopunk(soup, mydb):
    # Obtener el tipo de cryptopunk
    h4 = soup.find_all('h4')
    tipo_cp = h4[0].text.split()[3]
    print(tipo_cp)
    
    # Obtener el nombre del owner, y saber si está a la venta y su precio
    varios =soup.find_all('div', class_="col-md-10 col-md-offset-1")

    for linea in varios:
        texto = linea.text
        # Busco la línea que comienza con ...
        if texto.strip().startswith('Current Market Status') and texto.find('currently')>0:
            # Extraigo el nombre del dueño
            if texto.find("This punk is currently owned by address")>0:
                owner = texto.split('address ')[1].split('.',1)[0]
                print('Owner:',owner)
        
            # Averiguo si está en venta
            precio_venta = 0
            if texto.find("This punk is currently for sale by owner")>0:
                sale = True
                precio_venta = texto.split('by owner for ')[1].split('Ξ',1)[0]
                print('Precio de venta', precio_venta)
            else:
                sale = False # Dice 'This punk has not been listed for sale by its owner'
            print('For sale',sale)
        
            # Averiguo si hay ofertas
            oferta = 0
            if texto.find("There is a bid of")>0:
                bids = True
                oferta = texto.split('is a bid of ')[1].split('Ξ',1)[0]
                print('Valor ofrecido', oferta)
             
            else:
                bids = False  # Dice "There are currently no bids on this punk."
            print('Bids:',bids)

    # Obtain the typeID
    mycursor.execute("SELECT id FROM types WHERE name = %s", (tipo_cp.strip(),))
    id_tipo = mycursor.fetchone()[0]

    # Insert cryptopunk data into the table
    sql = "INSERT INTO cryptopunks (id, owner, id_type, for_sale, bids) VALUES (%s, %s, %s, %s, %s )"

    mycursor.execute(sql, (nro_cp, owner, id_tipo, sale, bids ))

    
    mydb.commit()        

## Función: Obtener la lista de transacciones realizadas

In [4]:
def obtener_transacciones(soup, mydb):
    
    def obtener_montos(str1):
        salida = re.search("(((\(\$)\s*\d+)([,]\d+){0,1}([,]\d+){0,1}([.]?\d{1,2})?)[MK]?\)", str1)
        con_coma = salida[0][2:-1]
        sin_coma = con_coma.replace(',','')
        if sin_coma.find('M')>0:
            sin_coma = sin_coma[0:-1]
            monto_dols = float(sin_coma)
            monto_dols = monto_dols * 1000000
        elif sin_coma.find('K')>0:
            sin_coma = sin_coma[0:-1]
            monto_dols = float(sin_coma)
            monto_dols = monto_dols * 1000
        else: 
            monto_dols = float(sin_coma)
        con_E = re.search("(\d+([,]\d+)?([.]\d{2})?)?\Ξ", str1)
        monto = float(con_E[1].replace(',',''))
        return monto, monto_dols
        
    text1 = soup.find_all('a', text = '')
    #print(text1)
    
    transacciones =[]
    
    transac =soup.find('table', class_="table")
    #print(transac)

    for tr in transac.find_all('tr'):
        lista = []
        for td in tr.find_all('td'):
            lista.append(td.text.strip())
        transacciones.append(lista)
    transacciones = transacciones[1:]
    print(transacciones)
    
  
    # Insert data into the table
    sql = "INSERT INTO transactions (id_cryptopunk, tr_type, tr_from, tr_to, amount, amount_dols, txn) VALUES (%s, %s, %s, %s, %s, %s, %s)"

    for tr in transacciones:
        # si la transaccion no esta vacia
        if len(tr)>0:
            # Hay un monto de transacción?
            monto = 0
            monto_dols = 0
            if len(tr[3])>0:
                monto, monto_dols = obtener_montos(tr[3])
            
            fecha = datetime.strptime(tr[4], "%b %d, %Y")
            
    
            mycursor.execute(sql, (nro_cp, tr[0],  tr[1], tr[2], monto, monto_dols, fecha))

    
    mydb.commit()

## Función: Obtener atributos del cryptopunk

In [5]:
def obtener_atributos(soup, mydb):
    cod =soup.find_all('div', class_="col-md-4")
    textos = []
    for c in cod:
        textos.append(c.find('a').text)
    
    atributos = textos[1:]
    print('Atributos:', atributos)
    
    # Insert data into the table attributes
    sql = "INSERT INTO cryptopunk_has_attributes (id_cryptopunk, id_attribute) VALUES (%s, %s)"
    

    for atr in atributos:
        # Obtain the typeID
        mycursor.execute("SELECT id FROM attributes WHERE name = %s", (atr.strip(),))
        id_atr = mycursor.fetchone()[0]
    
        mycursor.execute(sql, (nro_cp,id_atr))


    mydb.commit()

## Ejecución del proceso

In [ ]:
# Rango de cryptopunks a scrapear
for nro_cp in range(368,10000):
    
    # Define cryptopunk to scrap
    url = 'https://www.larvalabs.com/cryptopunks/details/'+str(nro_cp)

    # Use BeautifulSoup por webscrapping
    html=requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')

    obtener_cryptopunk(soup, mydb)
    obtener_transacciones(soup, mydb)
    obtener_atributos(soup, mydb)
    time.sleep(5)

Female
Owner: 0xd75d10
For sale False
Bids: False
[['Sold', '0x19e358', '0xd75d10', '70Ξ ($234,630)', 'Aug 23, 2021'], ['Offered', '', '', '70Ξ ($231,235)', 'Aug 23, 2021'], ['Transfer', '0x045f70', '0x19e358', '', 'Aug 07, 2021'], ['Transfer', '0x1cbb87', '0x045f70', '', 'Aug 05, 2021'], ['Sold', '0x471ccd', '0x1cbb87', '32Ξ ($78,811)', 'Jul 31, 2021'], ['Offered', '', '', '32Ξ ($78,698)', 'Jul 31, 2021'], ['Offer Withdrawn', '', '', '', 'Jul 31, 2021'], ['Offered', '', '', '28.99Ξ ($70,593)', 'Jul 30, 2021'], ['Offer Withdrawn', '', '', '', 'Jul 30, 2021'], ['Offered', '', '', '24.99Ξ ($54,790)', 'Jul 25, 2021'], ['Sold', 'punksotc…', '0x471ccd', '21.80Ξ ($47,102)', 'Jul 25, 2021'], ['Bid Withdrawn', '0x471ccd', '', '21.50Ξ ($46,375)', 'Jul 25, 2021'], ['Bid', '0x471ccd', '', '21.50Ξ ($45,727)', 'Jul 24, 2021'], ['Bid Withdrawn', '0x471ccd', '', '20.80Ξ ($44,238)', 'Jul 24, 2021'], ['Offered', '', '', '21.80Ξ ($43,439)', 'Jul 22, 2021'], ['Bid', '0x471ccd', '', '20.80Ξ ($41,422)', 'J

Female
Owner: 0x0845fc
For sale False
Bids: False
[['Transfer', '0xbb1ec7', '0x0845fc', '', 'Oct 12, 2021'], ['Sold', '0xfeeb2d', '0xbb1ec7', '39.70Ξ ($105,043)', 'Apr 27, 2021'], ['Bid Withdrawn', '0xbb1ec7', '', '33.30Ξ ($88,109)', 'Apr 27, 2021'], ['Bid', '0xbb1ec7', '', '33.30Ξ ($75,676)', 'Apr 24, 2021'], ['Offered', '', '', '39.70Ξ ($92,613)', 'Apr 21, 2021'], ['Transfer', '0xc352b5', '0xfeeb2d', '', 'Apr 21, 2021'], ['Claimed', '', '0xc352b5', '', 'Jun 23, 2017']]
Atributos: ['Green Eye Shadow', 'Straight Hair Blonde']
Female
Owner: 0x1437d2
For sale False
Bids: False
[['Sold', '0x7c5aee', '0x1437d2', '48.20Ξ ($87,099)', 'Mar 16, 2021'], ['Bid', '0x1437d2', '', '48.20Ξ ($85,721)', 'Mar 15, 2021'], ['Offered', '', '', '68Ξ ($127,152)', 'Mar 15, 2021'], ['Sold', 'Pranksy', '0x7c5aee', '24Ξ ($44,793)', 'Mar 14, 2021'], ['Bid', '0x7c5aee', '', '24Ξ ($44,355)', 'Mar 14, 2021'], ['Offered', '', '', '26.99Ξ ($48,128)', 'Feb 22, 2021'], ['Offered', '', '', '49.99Ξ ($95,037)', 'Feb 18, 2

Male
Owner: 0xc352b5
For sale False
Bids: False
[['Bid Withdrawn', 'daddykal…', '', '40Ξ ($104,007)', 'Jun 15, 2021'], ['Bid', 'daddykal…', '', '40Ξ ($96,146)', 'Jun 11, 2021'], ['Claimed', '', '0xc352b5', '', 'Jun 23, 2017']]
Atributos: ['Shaved Head']
Male
Owner: 0xb88f61
For sale False
Bids: False
[['Transfer', '0xc352b5', '0xb88f61', '', 'Feb 05, 2021'], ['Bid Withdrawn', '0x5a2598', '', '0.10Ξ ($18)', 'Nov 02, 2019'], ['Bid', '0x5a2598', '', '0.10Ξ ($18)', 'Nov 02, 2019'], ['Bid', 'Leduc', '', '<0.01Ξ ($1)', 'Nov 01, 2019'], ['Bid', '0xad2bf4', '', '<0.01Ξ ($2)', 'Jun 15, 2019'], ['Claimed', '', '0xc352b5', '', 'Jun 23, 2017']]
Atributos: ['Cigarette', 'Cap Forward', 'VR']
Female
Owner: 0xc352b5
For sale False
Bids: False
[['Bid Withdrawn', 'Pranksy', '', '4Ξ ($794)', 'May 22, 2020'], ['Bid', 'Pranksy', '', '4Ξ ($839)', 'May 20, 2020'], ['Bid Withdrawn', 'Pranksy', '', '1Ξ ($136)', 'Jan 05, 2020'], ['Bid', 'Pranksy', '', '1Ξ ($132)', 'Jan 01, 2020'], ['Bid', 'backseat…', '', '0.08